## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-29-01-15-31 +0000,startribune,Minnesota halts payments to housing program pr...,https://www.startribune.com/minnesota-halts-pa...
1,2025-07-29-01-04-43 +0000,nypost,345 Park Ave office building where Shane Tamur...,https://nypost.com/2025/07/28/us-news/345-park...
2,2025-07-29-01-03-37 +0000,nyt,Delta Pilot Is Arrested on Child Sex Abuse Cha...,https://www.nytimes.com/2025/07/28/business/de...
3,2025-07-29-01-02-49 +0000,nyt,"Heavy Rainfall in China Kills at Least 30, Off...",https://www.nytimes.com/2025/07/28/world/asia/...
4,2025-07-29-01-00-29 +0000,nypost,Witnesses recount panic during Shane Tamura’s ...,https://nypost.com/2025/07/28/us-news/witnesse...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2348/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,59
293,deal,21
292,trade,21
184,gaza,17
175,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
118,2025-07-28-18-24-16 +0000,nypost,France presses EU to unload ‘trade bazooka’ on...,https://nypost.com/2025/07/28/business/france-...,129
175,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...,129
137,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...,120
203,2025-07-28-11-56-36 +0000,nyt,"Europe, Worried by Other Global Crises, Agrees...",https://www.nytimes.com/2025/07/28/world/europ...,118
121,2025-07-28-18-21-50 +0000,nyt,"Trump and Starmer Expected to Discuss Trade, G...",https://www.nytimes.com/2025/07/28/us/politics...,115


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
118,129,2025-07-28-18-24-16 +0000,nypost,France presses EU to unload ‘trade bazooka’ on...,https://nypost.com/2025/07/28/business/france-...
137,61,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...
31,53,2025-07-28-23-11-27 +0000,nypost,Rifle-toting NYC shooter Shane Tamura kills 5 ...,https://nypost.com/2025/07/28/us-news/active-s...
142,47,2025-07-28-16-50-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...
204,43,2025-07-28-11-48-30 +0000,cbc,"Mother, newborn among at least 78 killed in Is...",https://www.cbc.ca/news/world/israel-gaza-aid-...
210,40,2025-07-28-11-00-00 +0000,wsj,Most Americans support President Trump’s immig...,https://www.wsj.com/politics/policy/trump-immi...
172,38,2025-07-28-15-04-00 +0000,wsj,Former Gov. Roy Cooper of North Carolina launc...,https://www.wsj.com/politics/elections/roy-coo...
57,34,2025-07-28-21-40-06 +0000,nypost,Heartbreaking video shows family’s tearful sen...,https://nypost.com/2025/07/28/us-news/family-t...
5,30,2025-07-29-01-00-00 +0000,wsj,Federal Reserve officials expect they will nee...,https://www.wsj.com/economy/central-banking/a-...
10,29,2025-07-29-00-37-55 +0000,nyt,Trump Administration Live Updates: President M...,https://www.nytimes.com/live/2025/07/28/us/tru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
